In [2]:
from IPython.display import HTML

HTML('''<script>
code_show=false; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Show code"></form>''')

In [3]:
import qgrid
qgrid.nbinstall(overwrite=True)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [4]:
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff

def make_gantt(schedule, start_date='2017-01-01'):        
    df = schedule.reset_index()
    
    for col in ['Start', 'Finish']:
        df[col] = df[col].apply(lambda x: (pd.Timedelta('%d days' % x) + pd.to_datetime(start_date)).date())
        
    df = df.to_dict('records')
    
    gantt = ff.create_gantt(df, showgrid_x=True, index_col='Resource', group_tasks=True, show_colorbar=True)
    
    for gantt_row, schedule_row in zip(gantt['data'], df):
        text = "No. batches produced: %d<br>Total duration: %d days" \
            % (schedule_row['Batches'], (schedule_row['Finish'] - schedule_row['Start']).days)
            
        gantt_row.update({'text': text})
        
    return gantt

In [5]:
num_usp_suites = 2
num_dsp_suites = 2

print("No. USP Suites: %d, no. DSP Suites: %d" % (num_usp_suites, num_dsp_suites))

No. USP Suites: 2, no. DSP Suites: 2


In [6]:
product_data = pd.read_csv('data/product_data.csv', index_col='Product')
qgrid.show_grid(product_data)

In [7]:
days_per_period = [60, 60, 60, 60, 60, 60]
demand_data = pd.read_csv('data/demand_data.csv', index_col='Product')
qgrid.show_grid(demand_data)

In [8]:
from lakhdar2005_continuous import Example1Model

example1_model = Example1Model().fit(
    num_usp_suites,
    num_dsp_suites,
    
    demand_data.values,
    days_per_period,
    
    product_data['USP storage cost'],
    product_data['Sales price'],
    product_data['Prod. cost'],
    product_data['Waste disp. cost'],
    product_data['DSP storage cost'],
    product_data['Backlog penalty'],
    product_data['Changeover cost'],
    
    product_data['USP rate [days]'],
    product_data['USP lead [days]'],
    product_data['USP lifetime [days]'],
    product_data['USP cap. per 60 days'],
    
    product_data['DSP rate [days]'],
    product_data['DSP lead [days]'],
    product_data['DSP lifetime [days]'],
    product_data['DSP cap. per 60 days']
)

100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:09<00:00, 1101.67it/s]


Best solution found -> Profit: 518.00, Backlog penalty: 0.00


In [10]:
gantt = make_gantt(example1_model.schedule)
py.iplot(gantt, filename='Lakhdar2005 Example1 Continuous-time Gantt', world_readable=True)

In [17]:
df = pd.DataFrame({key : [value] for key, value in example1_model.objectives.items()}, index=['value'])
df[['profit', 'sales', 'backlog_cost', 'production_cost', 'changeover_cost', 'dsp_storage_cost', 'dsp_waste_cost']]

,profit,sales,backlog_cost,production_cost,changeover_cost,dsp_storage_cost,dsp_waste_cost
value,518.0,680.0,0.0,136.0,10.0,16.0,0.0


In [13]:
example1_model.inventory_profile

array([[0, 0, 3, 0, 3, 0],
       [0, 3, 0, 0, 0, 0],
       [4, 0, 0, 4, 0, 0]])

In [18]:
example1_model.sales_profile

array([[0, 0, 0, 6, 0, 6],
       [0, 0, 6, 0, 0, 0],
       [0, 8, 0, 0, 8, 0]])

In [20]:
example1_model.backlog_profile

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])